In [2]:
import pandas as pd; pd.set_option('display.max_columns', 30); pd.set_option('display.max_rows', 100)
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from cmlreaders import get_data_index
from ripples_HFA_SCE import ripple_analysis_SCE
import statsmodels.formula.api as smf

In [3]:
def convert_list_to_string(list1):
    
    list_str = ''
    for i, val in enumerate(list1):
        list_str += str(val)
        if i + 1 < len(list1):
            list_str += '_'
            
    return list_str

# Set analysis mode, SME or SCE 
analyses_mode = 'SCE'

# init variables
sub_selection = 'first_half'
df = get_data_index("r1") # all RAM subjects
print(f"df {type(df)}")
exp = 'catFR1' # 'FR1' 'catFR1' 'RepFR1'
region_name = 'HPC'
regions_selected = ['ca1']
hpc_regions = ['ca1']
rs_str = ''
num_regions = len(regions_selected)
rs_str = convert_list_to_string(regions_selected)
hpc_ripple_type = 'single_elec'
select_subfield = True

df <class 'pandas.core.frame.DataFrame'>


In [4]:
data_folder = 'SWR_semantic_scratch'   
ripple_bin_start_end = [100, 1700] 
ripple_str = convert_list_to_string(ripple_bin_start_end)
ripple_delta = True
HFA_delta = False
select_subfield = True
formula_list = ['re~c', 'h~(c+r)*re', 'h~r+c']
re_formula_list = ['c', '(c+r)*re', 'r+c']
formula_idx = 0


RS = ripple_analysis_SCE(exp=exp, df=df, sub_selection=sub_selection, data_folder=data_folder, hpc_ripple_type=hpc_ripple_type,
                        select_subfield=select_subfield, hpc_regions=hpc_regions, regions_selected=regions_selected, 
                        ripple_bin_start_end=ripple_bin_start_end)

RS.remove_subject_sessions()
RS.load_data_from_cluster('encoding', region_name=region_name)
RS.getStartArray()
RS.select_idxs_numpy()
RS.create_clustered_array()
RS.recalled_data()

/home1/efeghhi/ripple_memory/ripples_HFA_analysis.py:116: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ((self.df.subject!='R1239E') | (self.df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)


catFR1
2023-05-16_02-41-52: DF Exception: Sub: R1147P, Sess: 0, FileNotFoundError, [Errno 2] No such file or directory: '/scratch/john/SWR_semantic_scratch/ENCODING/SWR_catFR1_R1147P_0_HPC_encoding_soz_in_hamming.p', file: ripples_HFA_analysis.py, line no: 195
2023-05-16_02-41-54: DF Exception: Sub: R1269E, Sess: 1, FileNotFoundError, [Errno 2] No such file or directory: '/scratch/john/SWR_semantic_scratch/ENCODING/SWR_catFR1_R1269E_1_HPC_encoding_soz_in_hamming.p', file: ripples_HFA_analysis.py, line no: 195
2023-05-16_02-41-54: DF Exception: Sub: R1269E, Sess: 3, FileNotFoundError, [Errno 2] No such file or directory: '/scratch/john/SWR_semantic_scratch/ENCODING/SWR_catFR1_R1269E_3_HPC_encoding_soz_in_hamming.p', file: ripples_HFA_analysis.py, line no: 195
2023-05-16_02-41-56: DF Exception: Sub: R1354E, Sess: 0, FileNotFoundError, [Errno 2] No such file or directory: '/scratch/john/SWR_semantic_scratch/ENCODING/SWR_catFR1_R1354E_0_HPC_encoding_soz_in_hamming.p', file: ripples_HFA_ana

In [5]:
print(f"Adjacent semantic: {RS.adjacent_semantic}")
print(f"Remote semantic: {RS.remote_semantic}")
print(f"Remote unclustered: {RS.remote_nonsemantic}")
print(f"Dead end: {RS.dead_ends}")

Adjacent semantic: 2456
Remote semantic: 2961
Remote unclustered: 2192
Dead end: 1354


In [12]:
ripple_array = RS.ripple_array_np
ripple_exists = RS.ripple_exists_np
ripple_no_exist = 1 - RS.ripple_exists_np
clustered = RS.clustered_array_np
recalled = RS.word_correct_array_np
sess = RS.session_name_array_np
subj = RS.subject_name_array_np
HFA = RS.HFA_mean

print(f"Number of trials: {HFA.shape[0]}")

re_recalled = RS.ripple_exists_re
re_n_recalled = RS.no_ripple_exists_re
c_recalled = RS.clustered_re
HFA_recalled = RS.HFA_re
sess_recalled = RS.sess_re
subj_recalled = RS.subj_rec

print(f"Number of trials recalled: {HFA_recalled.shape[0]}")
print(f"Number of trials clustered out all recalled trials: {np.sum(c_recalled)}")

Number of trials: 22536
(10666, 30)
Number of trials recalled: 10666
Number of trials clustered out all recalled trials: 5402.0


In [7]:
data = pd.DataFrame(data={'HFA':HFA, 'ripple_exists': ripple_exists, 'ripple_no_exists': ripple_no_exist,
                          'recalled': recalled, 'session':sess, 'clustered':clustered,
                        'subject':subj})

data_recalled = pd.DataFrame(data={'HFA':HFA_recalled, 'ripple_exists': re_recalled, 'clustered':c_recalled,
                                   'ripple_no_exists':re_n_recalled,'session':sess_recalled, 
                                    'subject':subj_recalled})

ValueError: Data must be 1-dimensional

In [52]:
vc = {'session':'0+session'}

In [53]:
model_lmm = smf.mixedlm('clustered~ripple_exists', data_recalled, groups="subject", vc_formula=vc, 
                                re_formula='ripple_exists')
model_fit_lmm = model_lmm.fit(method=["lbfgs"])
model_fit_lmm.summary()

/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2115: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


<class 'statsmodels.iolib.summary2.Summary'>
"""
                Mixed Linear Model Regression Results
======================================================================
Model:                 MixedLM     Dependent Variable:     clustered  
No. Observations:      22346       Method:                 REML       
No. Groups:            48          Scale:                  0.2378     
Min. group size:       15          Likelihood:             -15784.0059
Max. group size:       6885        Converged:              Yes        
Mean group size:       465.5                                          
----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                    0.362    0.027 13.246 0.000  0.309  0.416
ripple_exists                0.008    0.014  0.570 0.568 -0.020  0.036
subject Var                  0.028                                    
subject x ripple_exists Cov -0.001    0.004                           
ripple_exists Var            0.002    0.007                           
session Var                  0.006    0.015                           
======================================================================

"""

In [34]:

model_lmm = smf.mixedlm('HFA~recalled', data, groups="subject", vc_formula=vc, 
                                re_formula='recalled')
model_fit_lmm = model_lmm.fit(method=["lbfgs"])
model_fit_lmm.summary()

/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2115: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
================================================================
Model:               MixedLM   Dependent Variable:   HFA        
No. Observations:    22536     Method:               REML       
No. Groups:          19        Scale:                2.0033     
Min. group size:     240       Likelihood:           -39884.3400
Max. group size:     10500     Converged:            Yes        
Mean group size:     1186.1                                     
----------------------------------------------------------------
                       Coef.  Std.Err.   z   P>|z| [0.025 0.975]
----------------------------------------------------------------
Intercept               0.018    0.137 0.133 0.894 -0.250  0.287
recalled                0.140    0.038 3.659 0.000  0.065  0.216
subject Var             0.249                                   
subject x recalled Cov -0.002    0.022                          
recalled Var            0.012    0.004                          
session Var             0.140                                   
================================================================

"""

In [35]:
vc = {'session':'0+session'}
model_lmm = smf.mixedlm('HFA~clustered', data_recalled, groups="subject", vc_formula=vc, 
                                re_formula='clustered')
model_fit_lmm = model_lmm.fit(method=["lbfgs"])
model_fit_lmm.summary()

/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2115: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
=================================================================
Model:                MixedLM   Dependent Variable:   HFA        
No. Observations:     10666     Method:               REML       
No. Groups:           19        Scale:                1.9821     
Min. group size:      38        Likelihood:           -18837.4285
Max. group size:      6885      Converged:            Yes        
Mean group size:      561.4                                      
-----------------------------------------------------------------
                        Coef.  Std.Err.   z   P>|z| [0.025 0.975]
-----------------------------------------------------------------
Intercept                0.110    0.139 0.792 0.428 -0.163  0.383
clustered                0.155    0.047 3.271 0.001  0.062  0.247
subject Var              0.241                                   
subject x clustered Cov -0.004    0.010                          
clustered Var            0.001    0.001                          
session Var              0.149                                   
=================================================================

"""

In [99]:
vc = {'session':'0+session'}
model_lmm = smf.mixedlm('HFA~recalled*ripple_exists', data, groups="subject", vc_formula=vc, 
                                re_formula='recalled*ripple_exists')
model_fit_lmm = model_lmm.fit(method=["lbfgs"])
model_fit_lmm.summary()

/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2115: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Mixed Linear Model Regression Results
=====================================================================================
Model:                      MixedLM          Dependent Variable:          HFA        
No. Observations:           56412            Method:                      REML       
No. Groups:                 48               Scale:                       1.7612     
Min. group size:            192              Likelihood:                  -96165.5859
Max. group size:            10500            Converged:                   Yes        
Mean group size:            1175.2                                                   
-------------------------------------------------------------------------------------
                                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                  -0.164    0.019 -8.758 0.000 -0.201 -0.127
recalled                                    0.126    0.024  5.286 0.000  0.079  0.172
ripple_exists                               1.019    0.064 15.952 0.000  0.894  1.145
recalled:ripple_exists                     -0.096    0.041 -2.345 0.019 -0.177 -0.016
subject Var                                 0.007    0.003                           
subject x recalled Cov                      0.002    0.003                           
recalled Var                                0.012    0.005                           
subject x ripple_exists Cov                -0.008    0.006                           
recalled x ripple_exists Cov               -0.004    0.010                           
ripple_exists Var                           0.171    0.033                           
subject x recalled:ripple_exists Cov        0.003    0.005                           
recalled x recalled:ripple_exists Cov       0.014    0.005                           
ripple_exists x recalled:ripple_exists Cov -0.042    0.023                           
recalled:ripple_exists Var                  0.025    0.013                           
session Var                                 0.007    0.002                           
=====================================================================================

"""

In [100]:
vc = {'session':'0+session'}
model_lmm = smf.mixedlm('HFA~recalled*ripple_no_exists', data, groups="subject", vc_formula=vc, 
                                re_formula='recalled*ripple_no_exists')
model_fit_lmm = model_lmm.fit(method=["lbfgs"])
model_fit_lmm.summary()

/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2115: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Mixed Linear Model Regression Results
============================================================================================
Model:                         MixedLM            Dependent Variable:            HFA        
No. Observations:              56412              Method:                        REML       
No. Groups:                    48                 Scale:                         1.7608     
Min. group size:               192                Likelihood:                    -96177.9027
Max. group size:               10500              Converged:                     Yes        
Mean group size:               1175.2                                                       
--------------------------------------------------------------------------------------------
                                                 Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------------
Intercept                                         0.852    0.058  14.799 0.000  0.739  0.964
recalled                                          0.035    0.045   0.773 0.439 -0.053  0.122
ripple_no_exists                                 -1.016    0.063 -16.006 0.000 -1.140 -0.891
recalled:ripple_no_exists                         0.094    0.048   1.948 0.051 -0.001  0.188
subject Var                                       0.135    0.026                            
subject x recalled Cov                           -0.005    0.014                            
recalled Var                                      0.041    0.013                            
subject x ripple_no_exists Cov                   -0.143    0.029                            
recalled x ripple_no_exists Cov                   0.011    0.017                            
ripple_no_exists Var                              0.167    0.036                            
subject x recalled:ripple_no_exists Cov           0.029    0.018                            
recalled x recalled:ripple_no_exists Cov         -0.018    0.016                            
ripple_no_exists x recalled:ripple_no_exists Cov -0.044    0.019                            
recalled:ripple_no_exists Var                     0.041    0.011                            
session Var                                       0.005    0.001                            
============================================================================================

"""

In [36]:
vc = {'session':'0+session'}
model_lmm = smf.mixedlm('HFA~clustered*ripple_exists', data_recalled, groups="subject", vc_formula=vc, 
                                re_formula='clustered*ripple_exists')
model_fit_lmm = model_lmm.fit(method=["lbfgs"])
model_fit_lmm.summary()

/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2063: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2075: ConvergenceWarning: Gradient optimization failed, |grad| = 66.544925
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/statsmodels/regression/mixed

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Mixed Linear Model Regression Results
======================================================================================
Model:                       MixedLM          Dependent Variable:          HFA        
No. Observations:            10666            Method:                      REML       
No. Groups:                  19               Scale:                       1.8537     
Min. group size:             38               Likelihood:                  -18495.7416
Max. group size:             6885             Converged:                   No         
Mean group size:             561.4                                                    
--------------------------------------------------------------------------------------
                                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------
Intercept                                   -0.001    0.139 -0.009 0.993 -0.273  0.271
clustered                                    0.105    0.053  1.961 0.050  0.000  0.209
ripple_exists                                0.911    0.081 11.208 0.000  0.752  1.070
clustered:ripple_exists                     -0.179    0.172 -1.043 0.297 -0.515  0.157
subject Var                                  0.171                                    
subject x clustered Cov                     -0.005    0.028                           
clustered Var                                0.007    0.006                           
subject x ripple_exists Cov                 -0.003    0.028                           
clustered x ripple_exists Cov               -0.002    0.006                           
ripple_exists Var                            0.009    0.015                           
subject x clustered:ripple_exists Cov        0.006                                    
clustered x clustered:ripple_exists Cov      0.011    0.023                           
ripple_exists x clustered:ripple_exists Cov -0.014    0.006                           
clustered:ripple_exists Var                  0.174                                    
session Var                                  0.235                                    
======================================================================================

"""